In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from vllm import LLM, SamplingParams
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import numpy as np
import glob

/home/aicombined/miniconda3/envs/poc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-17 20:54:22,631	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


- Instruction Formatting 전처리 함수

In [2]:
def preprocess_text(x):
    return f"###입력:{x}\n\n###출력:"

- merge_16bit로 저장된 모델 weight의 폴더를 이용하여 llm를 로드

In [3]:
llm = LLM(model="huggingface_merged_weights/SOLAR/", enable_lora=True)

INFO 06-17 20:54:23 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='huggingface_merged_weights/SOLAR/', speculative_config=None, tokenizer='huggingface_merged_weights/SOLAR/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=huggingface_merged_weights/SOLAR/)
INFO 06-17 20:54:23 utils.py:660] Found nccl from library /home/aicombined/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 06-17 20:54:24 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
INFO 06-17 20:54:24 sele

- 자연어 생성을 위한 그리드 서치 디코딩 파라미터 선언

In [4]:
sampling_params = SamplingParams(max_tokens=256, temperature=0.5, top_k=50)

- korea_hate_speech 데이터셋 불러오기

In [5]:
sentences1 = []
sentences2 = []
labels = []

with open('./evaluation_dataset/03_kor_nli/xnli.test.ko.tsv', 'r') as f:
    kornli_data = f.readlines()
        
for data in kornli_data[1:]:
    data_split = data.split('\t')
    sentence1 = data_split[0]
    sentence2 = data_split[1]
    label = data_split[2].split('\n')[0]

    sentences1.append(sentence1)
    sentences2.append(sentence2)

    if label == 'contradiction':
        labels.append('모순')
    elif label == 'neutral':
        labels.append('중립')
    elif label == 'entailment':
        labels.append('함의')

In [6]:
sentences1[0], sentences2[0], labels[0]

('글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게 다시 이야기하게 되었다.',
 '나는 그와 다시 이야기하지 않았다.',
 '모순')

- 입력 데이터 전처리

In [10]:
system_prompt = """문장1과 문장 2의 관계에 대하여 함의 또는 중립 또는 모순으로 대답하세요. 문장 2가 논리적으로 문장 1을 따르는 경우 함의이고 문장 2와 문장 1의 논리적인 관계가 분명하지 않을 경우 중립이며 문장 2가 문장 1과 반대되는 경우는 모순입니다.\n\n"""
texts = []

for sentence1, sentence2, label in zip(sentences1, sentences2, labels):
    instruction = "문장 1 : {} \n 문장 2 : {}".format(sentence1, sentence2)
    texts.append(preprocess_text(system_prompt + instruction))

In [11]:
texts[0]

'###입력:문장1과 문장 2의 관계에 대하여 함의 또는 중립 또는 모순으로 대답하세요. 문장 2가 논리적으로 문장 1을 따르는 경우 함의이고 문장 2와 문장 1의 논리적인 관계가 분명하지 않을 경우 중립이며 문장 2가 문장 1과 반대되는 경우는 모순입니다.\n\n문장 1 : 글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게 다시 이야기하게 되었다. \n 문장 2 : 나는 그와 다시 이야기하지 않았다.\n\n###출력:'

- inference

In [12]:
outputs = llm.generate(texts, sampling_params)

Processed prompts: 100%|██████████| 5010/5010 [05:06<00:00, 16.37it/s]


- 생성 결과 전처리

In [13]:
preds = []
for output in outputs:
    generated_text = output.outputs[0].text
    preds.append(generated_text.split('$&%')[0])

In [14]:
preds_processed = []
labels_processed = []

for y_pred, y_true in zip(preds, labels):
    if y_pred == '모순' or y_pred == '중립' or y_pred == '함의':
        preds_processed.append(y_pred)
        labels_processed.append(y_true)

- zero-shot 분류 성능

In [15]:
print(classification_report(labels_processed, preds_processed))

              precision    recall  f1-score   support

          모순       0.49      0.95      0.65      1663
          중립       0.62      0.07      0.13      1614
          함의       0.59      0.53      0.56      1569

    accuracy                           0.52      4846
   macro avg       0.56      0.52      0.44      4846
weighted avg       0.56      0.52      0.44      4846

